# Comparação de Algoritmos de Aprendizado Supervisionado em Problemas de Classificação

- Gabriel Peres de Souza
- João Carlos dos Santos Correia
- Rodrigo Macedo Júnior

## Importando bibliotecas

In [ ]:
#     Importação das bibliotecas.

## Carregamento dos Dados

In [ ]:
#    Carregamento e descrição da base de dados.

## Pré-processamento dos dados (se precisar)

In [ ]:
# dados faltantes, transformação de dados categóricos em numéricos,
# escalas diferentes, outliers, dados desbalanceados, dados duplicados ou inconsistentes, ...

## Implementação dos algoritmos escolhidos

In [ ]:
# Descrição dos Algoritmos

## 1º Algoritmo

In [ ]:
# Implementação

## 2º Algorimo

In [ ]:
# Implementação

# Avaliação dos Modelos com métricas e grafos

In [ ]:
# Lembre-se de utilizar validação cruzada para divisão
# do conjunto de dados e métricas adequadas ao problema de classificação.

# Discussão dos resultados

In [ ]:
# Discussão

## Metodologia

**Algoritmos utilizados.** Implementamos dois classificadores:

1. **Regressão Logística treinada por Gradiente Descendente (SGDClassifier, loss='log\_loss')** — método linear, eficiente para bases grandes e ruidosas.
2. **k-Vizinhos Mais Próximos (k-NN)** — algoritmo não paramétrico que captura relações locais. Avaliamos diferentes valores de *k* e métrica de distância.

**Engenharia de atributos.** A partir de `Date` e `Time`, extraímos: ano, mês, dia, dia da semana, dia do ano, hora, minuto, minutos desde meia-noite e indicador de fim de semana; além de `Count`. O alvo é `Event` (0/1).

**Validação e métricas.** Usamos **validação cruzada k-fold com (k=10)** para uma análise mais precisa e profunda. Relatamos **Acurácia**, **Precisão**, **Sensibilidade (Recall)**, **F1**, **ROC** e **AUC**.

### Execução (scripts externos)

- `sgd_logreg.py`: análise com regressão logística (SGD)
- `knn_model.py`: análise com KNN (grid reduzido)
- `test_models.py`: roda tudo e salva JSONs
- `compare_plots.py`: gráficos comparativos (barras e ROC)

Artefatos em `/data`.

In [ ]:
%run /mnt/data/test_models.py
%run /mnt/data/compare_plots.py

In [ ]:
import json, pandas as pd
from IPython.display import Image, display
with open('/mnt/data/sgd_results.json') as f: sgd=json.load(f)
with open('/mnt/data/knn_results.json') as f: knn=json.load(f)
metrics=['accuracy','precision','recall','f1','roc_auc']
df=pd.DataFrame({'SGD (Logistic)':[sgd['metrics_point_on_full'][m] for m in metrics], 'KNN':[knn['metrics_point_on_full'][m] for m in metrics]}, index=metrics)
display(df.style.format('{:.4f}'))
display(Image(filename='/mnt/data/metrics_comparison.png'))
display(Image(filename='/mnt/data/roc_both.png'))